In [ ]:
from os import path

import geopandas as gpd
import osmnx as ox

from swiss_urban_trees import wms

In [ ]:
# wms args
# wms_url = "https://raster.sitg.ge.ch/arcgis/services/ORTHOPHOTOS_COLLECTION/MapServer/WMSServer?request=GetCapabilities&service=WMS"
wms_url = ""
version = "1.3.0"
layer_name = "22"
crs = "EPSG:2056"
res = 0.25  # must be in meters
tile_size = 1000  # in pixels
format = "image/tiff"
nodata = None

aoi = "Geneva, Switzerland"
dst_dir = "../data/raw/sitg-tiles"
dst_filepath = "../data/raw/sitg-orthophoto-2019-tiles/tiles.gpkg"

In [ ]:
if path.exists(aoi):
    aoi_gdf = gpd.read_file(aoi)
else:
    aoi_gdf = ox.geocode_to_gdf(aoi)
aoi_gdf = aoi_gdf.to_crs(crs)

In [ ]:
wms_dwd = wms.WMSDownloader(
    wms_url,
    str(layer_name),  # ACHTUNG: ensure layer name is a string
    dst_dir,
    res,
    crs,
    tile_size,
    format,
    nodata=nodata,
    print_func=print,
)
tile_gdf = wms_dwd.download_aoi(aoi_gdf["geometry"].iloc[0], keep_existing=True)

In [ ]:
tile_gdf.assign(
    **{"tile_filename": tile_gdf["tile_filepath"].apply(path.basename)}
).drop("tile_filepath", axis="columns").to_file(dst_filepath)